In [56]:
import os
import glob
import pickle

import numpy as np
import pandas as pd
from scipy import signal
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy import units as u
from astropy.time import Time

import matplotlib.pyplot as plt

In [2]:
resol = 0.1*u.second

In [61]:
pulsar = '1133+16'

In [62]:
files = sorted(glob.glob('./data_part/' + pulsar + os.sep +'*' + pulsar + '.log'))
print(files)

['./data_part/1133+16\\010618_21_N2_00_part_1133+16.log', './data_part/1133+16\\020618_21_N2_00_part_1133+16.log', './data_part/1133+16\\030618_21_N2_00_part_1133+16.log', './data_part/1133+16\\040618_21_N2_00_part_1133+16.log', './data_part/1133+16\\050618_21_N2_00_part_1133+16.log', './data_part/1133+16\\060618_21_N2_00_part_1133+16.log', './data_part/1133+16\\070618_20_N2_00_part_1133+16.log', './data_part/1133+16\\080618_20_N2_00_part_1133+16.log', './data_part/1133+16\\090618_20_N2_00_part_1133+16.log', './data_part/1133+16\\100618_20_N2_00_part_1133+16.log', './data_part/1133+16\\110618_20_N2_00_part_1133+16.log', './data_part/1133+16\\120618_20_N2_00_part_1133+16.log', './data_part/1133+16\\130618_20_N2_00_part_1133+16.log', './data_part/1133+16\\140618_20_N2_00_part_1133+16.log', './data_part/1133+16\\150618_20_N2_00_part_1133+16.log', './data_part/1133+16\\160618_20_N2_00_part_1133+16.log', './data_part/1133+16\\170618_20_N2_00_part_1133+16.log', './data_part/1133+16\\180618_2

In [63]:
full_data = []
for file in files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
    full_data.append(data)
block_data = []
for i in range(len(full_data)):
    block_data.append(full_data[i:i+5])
block_data = [i for i in block_data if len(i) == 5]

In [64]:
"""
Данные объеденяются в группы по пять дней, поскольку в случае объеденения в группы по 6 дней,
количество добавляемых в конец нулей, превышает количество точек на один день
(возможно это внесет искажения)
"""
dict_result  = {}
for block in block_data:
    mark = str(int(block[0][1].mjd)) + '-' + str(int(block[-1][1].mjd))
    joined_array = block[0][3]
    for i in range(1,len(block)):
        dt = block[i][1] - block[i-1][1] 
        zeros = np.zeros(int(round((dt.to(u.second)/resol).value, 0)))
        joined_array = np.append(joined_array, zeros)
        joined_array = np.append(joined_array, block[i][3])
    """
    Переводим число x в двоичную систему. 
    Предположим, получилось 101, количество цифр равно 3.
    Значит, наше число содержится между 100 и 1000. 
    Переводим 100 и 1000 в десятичную, и вычмсляем разность между ними и x.
    """
    n = len(joined_array)
    x = bin(n)
    x_high = '0b1' + '0'*(len(x[2:]))
    add_zeros_val = int(x_high, 2) - len(joined_array)
    joined_array = np.append(joined_array, np.zeros(add_zeros_val))
    
    fs = 10 # частотота дискретизации временных рядов
    f, Pxx_spec = signal.welch(joined_array, fs, 'flattop', 2048*10, scaling='spectrum')
    
    inc = 10000
    inter_point = max(f)*inc
    spl = InterpolatedUnivariateSpline(f, Pxx_spec)
    xs = np.linspace(0, max(f), inter_point) 
    spline = spl(xs)
    
    points = np.argwhere(np.diff(np.sign(spline - 0.3*np.max(spline)))).flatten()
    points = [i for i in points if xs[i] > 0.009]
    freq = []
    for i in range(1, len(points), 2):
        max_point = np.argmax(spline[points[i - 1]:points[i]])
        freq.append(xs[points[i - 1] + max_point])
    dict_result[mark] = pd.Series(freq)
res_table = pd.DataFrame(dict_result).T

c:\users\professional\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


In [65]:
res_table

,0,1,2,3,4
58270-58274,0.841217,1.682534,2.52375,3.364967,4.206384
58271-58275,0.841317,1.682534,2.52385,3.365067,4.206384
58272-58276,0.841317,1.682634,2.52375,3.365067,4.206384
58273-58277,0.841317,1.682534,2.52385,3.365167,4.206484
58274-58278,0.841317,1.682334,2.52395,3.364967,4.206084
58275-58279,0.841217,1.682434,2.52375,3.364967,4.206184
58276-58280,0.841317,1.682434,2.52385,3.365067,4.206284
58277-58281,0.841317,1.682534,2.52375,3.365067,4.206284
58278-58282,0.841217,1.682434,2.52375,3.364967,4.206084
58279-58283,0.841317,1.682534,2.52375,3.365067,4.206284


In [66]:
print(round(np.mean(res_table[0]), 6), '+-', round(np.std(res_table[0]), 6))

0.841278 +- 4.9e-05


[0.7230144602892058, 1.4457289145782914, 2.168743374867497, 2.891357827156543]
